In [56]:
# Install required packages
!pip install openai pinecone-client tiktoken

import time
import logging
import tiktoken
import functools

from typing import List
from openai import OpenAI
from datetime import datetime
from pinecone import Pinecone, ServerlessSpec
from concurrent.futures import ThreadPoolExecutor

TIMEOUT = 10      # Timeout for API calls
MAX_RETRIES = 3   # For API calls
BATCH_SIZE = 100  # For batch processing embeddings


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [57]:
# API Keys and Configuration
OPENAI_API_KEY = 'your-openai-api-key'
PINECONE_API_KEY = 'your-pinecone-api-key'
PINECONE_CLOUD = 'aws'
PINECONE_REGION = 'us-east-1'

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Constants
MAX_TOKENS = 225
PINECONE_INDEX_NAME = 'challenge-index'
EMBEDDING_MODEL = "text-embedding-3-small"
CHAT_MODEL = "gpt-4o"

# Create Pinecone index if it doesn't exist
try:
    index = pc.Index(PINECONE_INDEX_NAME)
except:
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            cloud=PINECONE_CLOUD,
            region=PINECONE_REGION
        )
    )
    index = pc.Index(PINECONE_INDEX_NAME)

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/home/prakhar/Documents/personal/jupiter_chatbot/env/lib/python3.11/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone


In [58]:
# Define the conversation history
history = [
    "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
    "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
    "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
    "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
    "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
    "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
    "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
    "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
    "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
    "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
    "11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
    "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
    "13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
    "14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
    "15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
    "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
    "17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
    "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
    "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
    "20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
]

In [59]:
def add_embeddings_to_pinecone(history: List[str], index_name: str = 'challenge-index') -> None:
    """add conversation history embeddings to vector db"""
    
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    
    @functools.lru_cache(maxsize=1000)
    def get_embedding(text: str) -> List[float]:
        for attempt in range(MAX_RETRIES):
            try:
                embedding = client.embeddings.create(
                    model=EMBEDDING_MODEL,
                    input=text,
                    timeout=TIMEOUT
                ).data[0].embedding
                return embedding
            except Exception as e:
                if attempt == MAX_RETRIES - 1:
                    logger.error(f"Failed to get embedding after {MAX_RETRIES} attempts: {e}")
                    raise
                time.sleep(2 ** attempt)
    
    # Process in batches
    for i in range(0, len(history), BATCH_SIZE):
        batch = history[i:i + BATCH_SIZE]
        vectors_to_upsert = []
        
        # Parallel embedding generation
        with ThreadPoolExecutor() as executor:
            embeddings = list(executor.map(get_embedding, batch))
        
        for message, embedding in zip(batch, embeddings):
            msg_num = message.split(':')[0]
            vector = {
                'id': f'msg_{msg_num}',
                'values': embedding,
                'metadata': {
                    'text': message,
                    'msg_num': int(msg_num),
                    'timestamp': datetime.utcnow().isoformat()
                }
            }
            vectors_to_upsert.append(vector)
        
        # Upsert
        try:
            index = pc.Index(index_name)
            index.upsert(vectors=vectors_to_upsert)
        except Exception as e:
            logger.error(f"Failed to upsert vectors: {e}")
            raise

    # Verify the index stats after upload
    try:
        index = pc.Index(index_name)
        stats = index.describe_index_stats()
    except Exception as e:
        logger.error(f"Failed to get index stats: {e}")

In [60]:
@functools.lru_cache(maxsize=100)

def retrieve_relevant_history(query: str, index_name: str = 'challenge-index', top_k: int = 5) -> List[str]:
    """retrieve relevant messages using hybrid search and reranking"""
    logger = logging.getLogger(__name__)
    
    try:
        index = pc.Index(index_name)
        
        # Get query embedding with retry logic
        query_embedding = client.embeddings.create(
            model=EMBEDDING_MODEL,
            input=query,
            timeout=TIMEOUT
        ).data[0].embedding
        
        # Hybrid search combining semantic and keyword matching
        results = index.query(
            vector=query_embedding,
            top_k=top_k * 2,
            include_metadata=True
        )

        # Rerank results using relevance scoring
        scored_results = []
        for match in results['matches']:
            # Calculate hybrid score
            semantic_score = match.score
            text_similarity = _calculate_text_similarity(query, match.metadata['text'])
            temporal_score = _calculate_temporal_score(match.metadata['timestamp'])
            
            final_score = (semantic_score * 0.6 + 
                         text_similarity * 0.3 + 
                         temporal_score * 0.1)
            
            scored_results.append((match, final_score))
        
        # Sort by final score and take top_k
        scored_results.sort(key=lambda x: x[1], reverse=True)
        relevant_messages = [match[0].metadata['text'] for match in scored_results[:top_k]]
        
        return relevant_messages
        
    except Exception as e:
        logger.error(f"Error retrieving history: {e}")
        return []

def _calculate_text_similarity(query: str, text: str) -> float:
    """Calculate simple text similarity score"""
    query_words = set(query.lower().split())
    text_words = set(text.lower().split())
    return len(query_words.intersection(text_words)) / len(query_words)

def _calculate_temporal_score(timestamp: str) -> float:
    """Calculate recency score"""
    time_diff = datetime.utcnow() - datetime.fromisoformat(timestamp)
    hours_old = time_diff.total_seconds() / 3600
    return max(0, 1 - (hours_old / 24))


In [61]:
def prepare_prompt(query: str, history: List[str], index_name: str = 'challenge-index'):
    """prepare the prompt for chatbot"""
    
    relevant_messages = retrieve_relevant_history(query, index_name)
    
    # init tokenizer
    encoding = tiktoken.encoding_for_model(CHAT_MODEL)
    
    # Extract and sort conversation pairs
    conversation_pairs = []
    for message in relevant_messages:
        parts = message.split(' | ')
        if len(parts) == 2:
            msg_num = int(parts[0].split(':')[0])
            conversation_pairs.append((msg_num, parts[0].split(': ', 1)[1], parts[1].split(': ', 1)[1]))
    
    conversation_pairs.sort(key=lambda x: x[0])
    
    # Format context while respecting token limit
    context_parts = []
    total_tokens = 0
    system_prompt = (
        "You are a helpful assistant. Use the context provided to give relevant and consistent responses. "
        "Keep responses concise."
    )
    
    # Count system prompt tokens
    total_tokens += len(encoding.encode(system_prompt))
    total_tokens += len(encoding.encode(query))
    

    for _, user, bot in conversation_pairs:
        pair_text = f"User: {user}\nAssistant: {bot}\n"
        pair_tokens = len(encoding.encode(pair_text))
        
        if total_tokens + pair_tokens > MAX_TOKENS - 50:
            break
            
        context_parts.append(pair_text)
        total_tokens += pair_tokens
    
    context = "\n".join(context_parts)
    
    messages = [
        {
            "role": "system",
            "content": f"{system_prompt}\n\nPrevious relevant conversation:\n{context}"
        },
        {"role": "user", "content": query}
    ]
    
    return messages, relevant_messages


In [62]:

def test_final_prompt():
    """test the final prompt"""
    
    final_test_prompt = "what was mountain name we discussed before?"
    
    # prepare prompt
    messages, context_referred = prepare_prompt(final_test_prompt, history)

    
    response = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=messages,
        max_tokens=100,
        temperature=0.7
    )

    bot_response = response.choices[0].message.content

    new_msg_num = len(history) + 1
    new_conversation = f"{new_msg_num}: User: {final_test_prompt} | Bot: {bot_response}"
    
    # add new conversation to vector db
    add_embeddings_to_pinecone([new_conversation])
    
    print(f"Final Test Prompt: {final_test_prompt}")
    print(f"Context Used: {context_referred}")
    print(f"Response: {bot_response}")

test_final_prompt()

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Final Test Prompt: what was mountain name we discussed before?
Context Used: ['11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.', '21: User: what lunch you have suggested me before? | Bot: I suggested trying a quinoa salad with avocado and chicken for lunch.', "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.", "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.", "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative."]
Response: We discussed the Rockies and the Appalachian Mountains.
